$ python3 gpt2-generate.py --model gpt2ja-medium --num_generate 1

In [1]:
#!pip install jaconv

In [2]:
model_path = r"E:\CloudSource\CloudSource_share\tharhtetsan\testing\gpt2-japanes\gpt2-japanese-master\gpt2ja-medium"

In [3]:
import json
import os
import numpy as np
import tensorflow.compat.v1 as tf
import argparse
from tqdm import tqdm
from sampling import sample_sequence
from  encode_bpe  import  BPEEncoder_ja

In [4]:
from model import HParams as HParams

In [5]:
with open('ja-bpe.txt', encoding='utf-8') as f:
    bpe = f.read().split('\n')

with open('emoji.json', encoding='utf-8') as f:
    emoji = json.loads(f.read())

In [6]:
enc  =  BPEEncoder_ja ( bpe , emoji )
n_vocab  =  len ( enc )

In [7]:
hparams = HParams(**{
      "n_vocab": n_vocab,
      "n_ctx": 1024,
      "n_embd": 1024,
      "n_head": 16,
      "n_layer": 24
    })

In [8]:
length=hparams.n_ctx // 2
temperature=1
top_k=40
top_p=0
min_length=max(0,0)
max_length = 500

input_context = "天井は普段あまり目のつかない箇所で、床などと違いこまめに掃除する方は少ないのではないでしょうか。 しかし、たまに天井の拭き掃除を行うと、想像以上に汚れていることに気づくでしょう。キッチン周辺の天井は油汚れがつきますし、喫煙者のいらっしゃるご家庭ではヤニが付着して変色します。 汚れのつきやすい環境に限らず日常生活で多少のホコリやゴミが発生しますので、これらが天井に付着することで自然と汚れていきます。ホコリなどは健康被害につながる可能性があるため、 住宅の安全性を保つ 上でも天井リフォームは重要です。"

In [9]:
def generate_one(sess, output):
    generated = ''
    
    
    pre_text = input_context if len(input_context)>0 else '<|endoftext|>'
    while True:
        
        print("###################################")
        print(pre_text)
        print("###################################")

        context_tokens = enc.encode(pre_text)
        if len(context_tokens) > length:
            context_tokens = context_tokens[-length:]
        out = sess.run(output, feed_dict={
            context: [context_tokens]
        })[:,len(context_tokens):]
        swd = enc.decode(out[0])
        last  =  False
        if '<|endoftext|>' in swd:
            swd = swd.split('<|endoftext|>')[0]
            last = True
        if len(swd) > 0:
            generated += swd

        if last or len(generated) > max_length:
            if len(generated) > 0:
                #print("generated : ",generated)
                return generated[:max_length]
        else:
            pre_text = generated[-256:]
        

In [10]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "0"

In [11]:
with tf.Session(config=config,graph=tf.Graph()) as sess:
    context = tf.placeholder(tf.int32, [1, None])
    output = sample_sequence(
        hparams=hparams, length=length,
        min_length=min_length, context=context,
        batch_size=1,
        temperature=temperature, top_k=top_k, top_p=top_p
    )
    
    saver = tf.train.Saver()
    
    ckpt = tf.train.latest_checkpoint(model_path)
    saver.restore(sess, ckpt)
    
    num_generate = 5
    for i in range(num_generate):
        output_result = generate_one(sess, output)
        print(str(output_result))
        if i < num_generate-1:
            print('========')
    
    

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.random.categorical` instead.
INFO:tensorflow:Restoring parameters from E:\CloudSource\CloudSource_share\tharhtetsan\testing\gpt2-japanes\gpt2-japanese-master\gpt2ja-medium\model-10410000
###################################
天井は普段あまり目のつかない箇所で、床などと違いこまめに掃除する方は少ないのではないでしょうか。 しかし、たまに天井の拭き掃除を行うと、想像以上に汚れていることに気づくでしょう。キッチン周辺の天井は油汚れがつきますし、喫煙者のいらっしゃるご家庭ではヤニが付着して変色します。 汚れのつきやすい環境に限らず日常生活で多少のホコリやゴミが発生しますので、これらが天井に付着することで自然と汚れていきます。ホコリなどは健康被害につながる可能性があるため、 住宅の安全性を保つ 上でも天井リフォームは重要です。
###################################

家中の天井の掃除をすると、ゴミやホコリがいっぱい、しかも壁の一部と、天井の一部が白くなってしまうわけにはいかない。天井にホコリがたまったらその分天井を拭き掃除して、カビが生えたらそこを拭き掃除。ホコリが溜まったらそこにホコリをまぶすことできれいになります。

 壁を磨く掃除の流れの中で、天井の汚れがたまっていくと大変危険なのは天井交換。「そのままでは天井にホコリがうち込むだけ」となり、天井を少しでも長く磨きたければ「天井にホコリがつき放題になることを防ぐため、白くなっている部分を掃除」することです。

 天井と天井が、少しずつ白い状態から黒の状態に戻っていくにつれて、そこに発生するホコリやカビが多くの場合に「カビ」となり、カビがつき放題になった部分はどんどん天井を白く染めていきますよ。カビやホコリは、カビがつき放題になったときに取れる菌菌の一つで、カビ予防として床や天井に漂い出て広がりやすくなる